# How to dispatch custom callback events

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Callbacks](/docs/concepts/#callbacks)
- [Custom callback handlers](/docs/how_to/custom_callbacks)
- [Astream Events API](/docs/concepts/#astream_events) the `astream_events` method will surface custom callback events.
:::

In some situations, you may want to dipsatch a custom callback event from within a [Runnable](/docs/concepts/#runnable-interface) so it can be surfaced
in a custom callback handler or via the [Astream Events API](/docs/concepts/#astream_events).

For example, if you have a long running tool with multiple steps, you can dispatch custom events between the steps and use these custom events to monitor progress.
You could also surface these custom events to an end user of your application to show them how the current task is progressing.

To dispatch a custom event you need to decide on two attributes for the event: the `name` and the `data`.

| Attribute | Type | Description                                                                                              |
|-----------|------|----------------------------------------------------------------------------------------------------------|
| name      | str  | A user defined name for the event.                                                                       |
| data      | Any  | The data associated with the event. This can be anything, though we suggest making it JSON serializable. |


:::{.callout-important}
* Dispatching custom callback events requires `langchain-core>=0.2.15`.
* Custom callback events can only be dispatched from within an existing `Runnable`.
* If using `astream_events`, you must use `version='v2'` to see custom events.
* Sending or rendering custom callbacks events in LangSmith is not yet supported.
:::


:::caution COMPATIBILITY
LangChain cannot automatically propagate configuration, including callbacks necessary for astream_events(), to child runnables if you are running async code in python<=3.10. This is a common reason why you may fail to see events being emitted from custom runnables or tools.

If you are running python<=3.10, you will need to manually propagate the `RunnableConfig` object to the child runnable in async environments. For an example of how to manually propagate the config, see the implementation of the `bar` RunnableLambda below.

If you are running python>=3.11, the `RunnableConfig` will automatically propagate to child runnables in async environment. However, it is still a good idea to propagate the `RunnableConfig` manually if your code may run in other Python versions.
:::

In [1]:
# | output: false
# | echo: false

%pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.


## Astream Events API

The most useful way to consume custom events is via the [Astream Events API](/docs/concepts/#astream_events).

We can use the `async` `adispatch_custom_event` API to emit custom events in an async setting. 


:::{.callout-important}

To see custom events via the astream events API, you need to use the newer `v2` API of `astream_events`.
:::

In [2]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def foo(x: str) -> str:
    await adispatch_custom_event("event1", {"x": x})
    await adispatch_custom_event("event2", 5)
    return x


async for event in foo.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'foo', 'tags': [], 'run_id': '42b30f10-7a2e-4b20-8843-bf17607cf0e0', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '42b30f10-7a2e-4b20-8843-bf17607cf0e0', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': '42b30f10-7a2e-4b20-8843-bf17607cf0e0', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '42b30f10-7a2e-4b20-8843-bf17607cf0e0', 'name': 'foo', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': '42b30f10-7a2e-4b20-8843-bf17607cf0e0', 'name': 'foo', 'tags': [], 'metadata': {}, 'parent_ids': []}


In python <= 3.10, you must propagate the config manually!

In [3]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async for event in bar.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'bar', 'tags': [], 'run_id': 'a5b44391-e5e1-4050-9f2b-9681a778ea05', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'a5b44391-e5e1-4050-9f2b-9681a778ea05', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'a5b44391-e5e1-4050-9f2b-9681a778ea05', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'a5b44391-e5e1-4050-9f2b-9681a778ea05', 'name': 'bar', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'a5b44391-e5e1-4050-9f2b-9681a778ea05', 'name': 'bar', 'tags': [], 'metadata': {}, 'parent_ids': []}


## Async Callback Handler

You can also consume the dispatched event via an async callback handler.

In [4]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class AsyncCustomCallbackHandler(AsyncCallbackHandler):
    async def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async_handler = AsyncCustomCallbackHandler()
await foo.ainvoke(1, {"callbacks": [async_handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 8b27fd9b-9961-4f33-94c7-65a6473322ae
Received event event2 with data: 5, with tags: ['foo', 'bar'], with metadata: {} and run_id: 8b27fd9b-9961-4f33-94c7-65a6473322ae


1

## Sync Callback Handler

Let's see how to emit custom events in a sync environment using `dispatch_custom_event`.

You **must** call `dispatch_custom_event` from within an existing `Runnable`.

In [5]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.callbacks.manager import (
    dispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class CustomHandler(BaseCallbackHandler):
    def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
def foo(x: int, config: RunnableConfig) -> int:
    dispatch_custom_event("event1", {"x": x})
    dispatch_custom_event("event2", {"x": x})
    return x


handler = CustomHandler()
foo.invoke(1, {"callbacks": [handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 5646baf3-1774-4b48-b941-2d051093916a
Received event event2 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 5646baf3-1774-4b48-b941-2d051093916a


1

## Next steps

You've seen how to emit custom events, you can check out the more in depth guide for [astream events](/docs/how_to/streaming/#using-stream-events) which is the easiest way to leverage custom events.